In [ ]:
import numpy as np

from a_nice_mc.models.discriminator import MLPDiscriminator
from a_nice_mc.models.generator import create_nice_network
from a_nice_mc.train.wgan_nll import Trainer

from hepmc.core.densities.qcd import ee_qq_ng
from hepmc.core.phase_space.mapping import MappedDensity
from hepmc.core.phase_space.rambo import RamboOnDiet

In [ ]:
import tensorflow as tf
from a_nice_mc.objectives import Energy
from tensorflow.python.framework import ops

class QCDTarget(Energy):
    def __init__(self):
        super(QCDTarget, self).__init__()
        self.name = "QCDTarget"
        self.z = tf.placeholder(tf.float32, [None, 8], name='z')
        target = ee_qq_ng(2, 100., 5., .3)
        rambo_mapping = RamboOnDiet(100., 4)
        self.mapped = MappedDensity(target, rambo_mapping)
    
    def __call__(self, z):
        return self.tf_energy(z)
    
    # energy as numpy function
    def energy(self, z):
        return self.mapped.pot(z).astype(np.float32, copy=False)
        #return self.camel.pot(z).view('float32')
    
    # energy as tensorflow function
    def tf_energy(self, z, name=None):
        with tf.name_scope(name, "energy", [z]) as name:
            y = tf.py_func(self.energy,
                       [z],
                       [tf.float32],
                       name=name,
                       stateful=False)
            return y[0]
    
    #@staticmethod
    #def mean():
    #    return np.array(3*[.5])
    
    #@staticmethod
    #def std():
    #    return np.array(3*[.181])
    
    def evaluate(self, zv, path=None):
        pass

In [ ]:
energy_fn = QCDTarget()

In [ ]:
#def noise_sampler(bs):
#    return np.random.normal(0.0, 1.0, [bs, 8])

def noise_sampler(bs):
    return np.random.uniform(0.0, 1.0, [bs, 8])

#def noise_sampler(bs):
#    noise = np.empty((bs, 8))
#    
#    indices = np.arange(bs)
#    while indices.size > 0:
#        x = np.random.normal(.5, .25, [indices.size, 8])
#        y = energy_fn.mapped.pot(x)
#        in_bounds = np.logical_not(np.isinf(y))
#        noise[indices[in_bounds]] = x[in_bounds]
#        indices = indices[np.logical_not(in_bounds)]
#
#    return noise

In [ ]:
discriminator = MLPDiscriminator([400, 400, 400])
generator = create_nice_network(
    8, 10,
    [
        ([400], 'v1', False),
        ([400, 400], 'x1', True),
        ([400], 'v2', False)
    ]
)

In [ ]:
trainer = Trainer(generator, energy_fn, discriminator, noise_sampler, b=16, m=4)

In [ ]:
trainer.train(max_iters=10000)

In [ ]:
sample, _ = trainer.sample(batch_size=32, steps=2000)

In [ ]:
burn_in = 1000
sample = sample[:, burn_in:]

In [ ]:
sample = np.reshape(sample, (-1, 8))

In [ ]:
n_accepted = 1
for i in range(1, 32000):
    if (sample[i] != sample[i-1]).any():
        n_accepted += 1
print(n_accepted)

In [ ]:
from hepmc.core.sampling import Sample
from hepmc.core.densities.qcd import export_hepmc
rambo_mapping = RamboOnDiet(100., 4)
anicemc_sample = Sample(data=rambo_mapping.map(sample), weights=np.full(sample.shape[0], 1./sample.shape[0]))
export_hepmc(100., anicemc_sample, "../samples/qcd/2-4/a_nice_mc.hepmc")

In [ ]:
sample_container = Sample(data=sample[0][0])

In [ ]:
mean = sample_container.mean
variance = sample_container.variance

In [ ]:
from hepmc.core.util.stat_tools import effective_sample_size

In [ ]:
effective_sample_size(sample_container, mean, variance)